In [ ]:
pip install openai

In [4]:
from openai import OpenAI

client = OpenAI (
    api_key ="API"
)



In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')
training_file_name = '/content/drive/MyDrive/HasoubAI/Finetune_Gpt-3.5-turbo/training_data.jsonl'
validation_file_name = '/content/drive/MyDrive/HasoubAI/Finetune_Gpt-3.5-turbo/validation_data.jsonl'




Mounted at /content/drive


In [ ]:
training_file_id = client.files.create(

    file = open(training_file_name, "rb"),
    purpose = "fine-tune"
)

validation_file_id = client.files.create(
    file = open(validation_file_name, "rb"),
    purpose = "fine-tune"

)


print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")




In [ ]:
response =client.fine_tuning.jobs.create(
    trainin_file = training_file_id.id,
    validation_file= validation_file_id.id,
    model = "gpt-3.5-turpo",
    hyperparameters={
        "n_epochs": 15,
        "batch_size": 3,
        "learning_rate_multiplier": 0.3,

    }
)
job_id = response
status = response.status

print(f'Fine-tunning model with jobID:{job_id}.')
print(f'Training Response:{response}')
print(f'Training Statud : {status}')




In [2]:
import signal
import datetime

def signal_handler(sig, frame):
  status = client.fine_tuning.jobs.retrieve(job_id).status
  print(f"Stream interrupted. Job is still {status}.")
  return

print(f"Stramiing events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGNAL, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tunning_job_id=job_id)
try:
  for event in events:
    print(
        f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
    )
except Exception:
  print("Stream interrupted (client disconnected).")






SyntaxError: incomplete input (<ipython-input-2-bd9f14e1c146>, line 16)

In [ ]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded" , "failed"]:
  print(f"Job not in terminal status: {status}. Waiting.")
  while status not in ["succedeed" , "failed"]:
    time.sleep(2)
    status = client.fine_tuning.jobs.retrive(job_id).status
    print(f"Status:{status}")

else:
  print(f"Finetune job {job_id} finished with status: {status}")





In [ ]:
result =  client.fine_tuning.jobs.list()
print(f"Found{len(result.data)}finetune jobs.")


In [ ]:
fine_tuned_model = result.data[0].fine_tuned_model

In [ ]:
print(fine_tuned_model)


In [ ]:
answer = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    message=[
        {"role":"user","content": "ماهي سياسه ارجاع الكتب لديكم؟"}

    ]

)

print(answer.choices[0].message)

In [ ]:
answer = client.chat.completions.create(
    model = fine_tuned_model,
    message=[
        {"role":"user","content": "ماهي سياسه ارجاع الكتب لديكم؟"}

    ]

)

print(answer.choices[0].message)